# Fine-tuning a BERT model for text extraction with the SQuAD dataset

We are going to fine-tune [BERT implemented by HuggingFace](https://huggingface.co/bert-base-uncased) for the text-extraction task with a dataset of questions and answers with the [SQuAD (The Stanford Question Answering Dataset)](https://rajpurkar.github.io/SQuAD-explorer/) dataset.
The data is composed by a set of questions and corresponding paragraphs that contains the answers.
The model will be trained to locate the answer in the context by giving the positions where the answer starts and ends.

In this notebook we are going to do the training using a single GPU.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

More info:
- [Glossary - HuggingFace docs](https://huggingface.co/transformers/glossary.html#model-inputs)
- [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [2]:
import numpy as np
import os
import json
import dataset_utils as du
import eval_utils as eu
import torch
from datetime import datetime
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from tokenizers import BertWordPieceTokenizer
from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm.notebook import tqdm

In [3]:
hf_model = 'bert-base-uncased'
bert_cache = os.path.join(os.getcwd(), 'cache')

In [4]:
slow_tokenizer = BertTokenizer.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'_{hf_model}-tokenizer')
)
save_path = os.path.join(bert_cache, f'{hf_model}-tokenizer')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)
    
# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(os.path.join(save_path, 'vocab.txt'),
                                   lowercase=True)

In [5]:
model = BertForQuestionAnswering.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'{hf_model}_qa')
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [6]:
train_path = os.path.join(bert_cache, 'data', 'train-v1.1.json')
eval_path = os.path.join(bert_cache, 'data', 'dev-v1.1.json')
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

In [8]:
max_len = 384

train_squad_examples = du.create_squad_examples(raw_train_data, max_len, tokenizer)
x_train, y_train = du.create_inputs_targets(train_squad_examples, shuffle=True, seed=42)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = du.create_squad_examples(raw_eval_data, max_len, tokenizer)
x_eval, y_eval = du.create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

86136 training points created.
10331 evaluation points created.


In [9]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        return (torch.tensor(self.x[0][idx]),
                torch.tensor(self.x[1][idx]),
                torch.tensor(self.x[2][idx]),
                torch.tensor(self.y[0][idx]),
                torch.tensor(self.y[1][idx]))

    def __len__(self):
        return len(self.x[0])

In [10]:
batch_size = 16

train_set = SquadDataset(x_train, y_train)
train_loader = DataLoader(train_set, batch_size=batch_size,
                          shuffle=True)

In [11]:
device = 0
model.to(device)
model.train()

model.training

True

In [12]:
optim = AdamW(model.parameters(), lr=5e-5)

In [13]:
for epoch in range(1):
    for i, batch in tqdm(enumerate(train_loader)):
        optim.zero_grad()
        outputs = model(input_ids=batch[0].to(device),
              token_type_ids=batch[1].to(device),
              attention_mask=batch[2].to(device),
              start_positions=batch[3].to(device),
              end_positions=batch[4].to(device)
             )
        
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        # if i > 10:
        #    break

0it [00:00, ?it/s]

## Save the model and load it on the cpu for evaluation

In [14]:
model_hash = datetime.now().strftime("%Y-%m-%d-%H%M%S")
model_path_name = './cache/model_trained_single_node_{model_hash}'

# save model's state_dict
torch.save(model.state_dict(), model_path_name)

# create the model again since the previous one is on the gpu
model_cpu = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased",
    cache_dir=os.path.join(bert_cache, 'bert-base-uncased_qa')
)

# load the model on cpu
model_cpu.load_state_dict(
    torch.load(model_path_name,
               map_location=torch.device('cpu'))
)

# load the model on gpu
# model.load_state_dict(torch.load(model_path_name))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

<All keys matched successfully>

In [18]:
model.eval()

model.training

False

In [16]:
samples = np.random.choice(len(x_eval[0]), 50, replace=False)

eu.EvalUtility(
    (x_eval[0][samples], x_eval[1][samples], x_eval[2][samples]),
    model_cpu,
    eval_squad_examples[samples]
).results()

                                         Evaluation                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                   Question ┃                  Model's answer ┃ Reference                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    How long was the Summer │               from 1870 to 1939 │  1870 to 1939              │
│      Theatre in operation? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│     Excessive bureaucratic │                      extralegal │  extra-legal               │
│     red tape is one of the │                                 │                            │
│   reasons for what type of │                                 │                            │
│                 ownership? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│    What company did Eisner │              paramount pictures │  Paramount Pictures        │
│   become president of when │                                 │                            │
│       he left ABC in 1976? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│      How long is the Upper │                          300 km │  300 km long               │
│               Rhine Plain? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│       What did Biraben say │  plague was present somewhere i │  the plague was present    │
│        about the plague in │                                 │ somewhere in Europe in     │
│                    Europe? │                                 │ every year between 1346    │
│                            │                                 │ and 1671.                  │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│         How long did Julia │  180foot 55 mtall 600yearold ca │  738 days                  │
│   Butterfly Hill live in a │                                 │                            │
│                      tree? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│        Who did Alexander I │             sybilla of normandy │  Sybilla of Normandy       │
│                     marry? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│        What remote control │        wireless controlled boat │  boat                      │
│       vehicle did he make? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│       What is Doctor Who's │                          tardis │  TARDIS                    │
│         space ship called? │                                 │                            │
├────────────────────────────┼─────────────────────────────────┼────────────────────────────┤
│    What is an example of a │              trypanosoma brucei │  Trypanosoma brucei        │
│     parasite that used the │                                 │                            │
│        antigenic variation │                                 │                            │
│          strategy to evade │                                 │                            │
│               destruction? │                                 │                            │
├────────────────────────────┼──────────────────────